# Falta fazer
## Relatório
## Documentação

In [1]:
# import das bibliotecas
import os
import pandas as pd
import random
import time
from greedy_filter import *
from math import inf
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model

In [2]:
# definicao das constantes
PATH_EXCEL = os.getcwd() + '/excel_files/'

Estruturas que irao armazenar os dataframes com o histórico de preços das ações.
1. dict_excels: dicionario onde a chave é o nome da ação e o valor da chave é o dataframe
2. filenames: lista com o nome dos arquivos
3. excels: lista com os dataframes

In [3]:
dict_excels = {}
filenames = []
excels = []
for filename in os.listdir(PATH_EXCEL):
    filenames.append(filename[:filename.find('.')])
    excels.append(pd.read_excel(PATH_EXCEL + filename).sort_values(by=['Exchange Date']).reset_index(drop=True))
    dict_excels[filename[:filename.find('.')]] = (pd.read_excel(PATH_EXCEL + filename).sort_values(by=['Exchange Date']).reset_index(drop=True))

In [4]:
filenames[6]

'alphabet'

## Modelagem do problema
### Algoritmo Genético 1 - Escolha dos Parâmetros das Regras de Filtro
1. Gene: cada um dos parâmetros do filtro
    1. x - porcentagem de variação: porcentagem acima/abaixo da ultima subida ou queda
       1. varia de 0 a 1
       2. varia de 0.01 a 0.10
    2. h - hold days: após um sinal de compra/venda esperamos por h dias
       1. varia de 1 até o número de dias entre a data atual e a data onde acabam as informacoes
       2. varia de 1 até 30
    3. d - delay days: após receber um sinal ignora-se os próximos d dias
       1. varia de 1 até o número de dias entre a data atual e a data onde acabam as informacoes
       2. varia de 1 até 30
    4. p - previous days: olha-se p dias para tomar uma decisão
       1. varia de 0 até o número de dias transcorridos até o momento
       2. varia de 30 até o número máximo de dias
2. Cromossomo: conjunto de parâmetros
    1. Estrutura usada: lista com os parâmetros &rarr; [x, h, d, p]
3. População: conjunto de cromossomos
    1. Tamanho da População:
4. Mutação:
    1. taxa de mutação:
5. Crossover:
    1. taxa de crossover:
6. Critério de Parada:
7. Seleção:
    1. Fitness: cálculo do lucro obtido por cada cromossomo
    2. Técnica de Seleção:
        1. Técnica 1:
        2. Técnica 2:

Funções para realizar o crossover entre dois cromossomos

In [5]:
def crossover(chromosome1, chromosome2):
    """
        Funcao para realizar o crossover entre dois cromossomos
        :param: chromosome1 - cromossomo pai
        :param: chromosome2 - cromossomo mae
        :return: new_chromosome1 - primeiro filho gerado no crossover
        :return: new_chromosome2 - segundo filho gerado no crossover
    """
    # escolhe um gene aleatorio para realizar o crossover
    rand = random.randint(1,len(chromosome1) - 1)

    # gera os dois novos cromossomos
    new_chromosome1 = chromosome1[:rand] + chromosome2[rand:]
    new_chromosome2 = chromosome2[:rand] + chromosome1[rand:]
    
    return new_chromosome1, new_chromosome2

def crossover2(chromosome1, chromosome2):
    """
        Funcao para realizar o crossover entre dois cromossomos com 2 pontos de corte
        :param: chromosome1 - cromossomo pai
        :param: chromosome2 - cromossomo mae
        :return: new_chromosome1 - primeiro filho gerado no crossover
        :return: new_chromosome2 - segundo filho gerado no crossover
    """
    # escolhe um gene aleatorio para realizar o crossover
    rand = random.randint(1,len(chromosome1) - 2)
    rand2 = random.randint(rand+1, len(chromosome1) - 1)
    # gera os dois novos cromossomos
    new_chromosome1 = chromosome1[:rand] + chromosome2[rand:rand2] + chromosome1[rand2:]
    new_chromosome2 = chromosome2[:rand] + chromosome1[rand:rand2] + chromosome2[rand2:]
    
    return new_chromosome1, new_chromosome2

Funções para realizar a mutação de um cromossomo

In [6]:
def mutation(chromossome):
    """
        Funcao para realizar a mutacao de um dado cromossomo
        :param: chromossome - cromossomo que ira passar pelo processo de mutacao
        :param: today_index - indice da tabela referente a data atual da acao
        :param: last_day_index - indice da tabela referente a ultima data da qual se tem informacao da acao
        :return: new_chromossome - novo cromossomo apos o processo de mutacao
    """
    # faz a copia do cromossomo original
    new_chromossome = chromossome
    
    # escolhe, aleatoriamente, um gene para ser alterado
    gene_initial_position = random.randint(0,len(chromossome) - 1)
    gene_final_position = random.randint(gene_initial_position, len(chromossome) - 1)
    
    # modifica o conjunto de genes escolhidos seguindo, como unica regra, os valores que eles podem assumir
    for i in range(gene_initial_position, gene_final_position + 1):
        if i == 0:
            new_chromossome[0] = random.uniform(0.01,0.10)
        elif i == 1:
            new_chromossome[1] = random.randint(1,30)
        elif i == 2:
            new_chromossome[2] = random.randint(1,30)
        elif i == 3:
            new_chromossome[3] = random.randint(30,500)
        else:
            raise Exception('Gene inexistente no cromossomo!')
        
    return new_chromossome

def mutation_v2(chromossome):
    """
        Funcao para realizar a mutacao de um dado cromossomo
        :param: chromossome - cromossomo que ira passar pelo processo de mutacao
        :return: new_chromossome - novo cromossomo apos o processo de mutacao
    """
    # faz a copia do cromossomo original
    new_chromossome = chromossome
    
    # escolhe, aleatoriamente, um gene para ser alterado
    gene_position = random.randint(0,len(chromossome) - 1)
    
    # modifica o gene escolhido seguindo, como unica regra, os valores que ele pode assumir
    if gene_position == 0:
        new_chromossome[0] = random.uniform(0.01,0.10)
    elif gene_position == 1:
        new_chromossome[1] = random.randint(1,30)
    elif gene_position == 2:
        new_chromossome[2] = random.randint(1,30)
    elif gene_position == 3:
        new_chromossome[3] = random.randint(30,500)
    else:
        raise Exception('Gene inexistente no cromossomo!')
        
    return new_chromossome

Função para realizar a inicialização randômica de uma população

In [7]:
def create_population(population_size):
    """
        Funcao para criar uma populacao randomica de cromossomos
        :param: population_size - tamanho da populacao que sera criada
        :return: population - nova populacao
    """
    population = []
    for i in range(0, population_size):
        population.append([random.uniform(0,0.1), random.randint(1,30), random.randint(1,30), random.randint(1,500)])
        
    return population

In [8]:
def fitness(np_array, chromossomes, budget):
    """
        Funcao para calcular o lucro de cada um dos cromossomos de uma dada populacao
        :param: np_array - conjunto de dados de determinada acao
        :param: chromossomes - lista de cromossomos que sera avaliada
        :param: budget - dinheiro inicial do problema
        :return: fit_chromossomes - matriz com os cromossomos e o lucro(em porcentagem) obtido por eles
    """
    fit_chromossomes = []
    for chromossome in chromossomes:
        money = greedy_filter_rule(np_array, chromossome, budget)
        fit_chromossomes.append([chromossome, (money-budget)/budget])
        
    return fit_chromossomes

In [9]:
def selection(stock_value, list_chromossomes, budget, cut_size):
    """
        :param: stock_value - 
        :param: list_chromossomes - 
        :param: budget - 
        :param: cut_size - 
        
        :return: new_generation - 
        :return: fitness_array - 
    """
    fitness_array = fitness(stock_value, list_chromossomes, budget)
    fitness_array.sort(key=lambda x: x[1], reverse = True)
    new_generation = []
    for i in range (0,cut_size):
        new_generation.append(fitness_array[i][0])
    return new_generation, fitness_array[:cut_size]

In [10]:
def roulette_selection(stock_value, list_chromossomes, budget, cut_size):
    """
        :param: stock_value - 
        :param: list_chromossomes - 
        :param: budget - 
        :param: cut_size - 
        
        :return: new_population - 
        :return: cut_size - 
    """
    fitness_array = fitness(stock_value, list_chromossomes, budget)
#     fitness_array = linear_normalization(fitness_array)

    adds_skills = 0
    for fit in fitness_array:
        adds_skills = adds_skills + fit[1]
    
    new_population = []
    for i in range(cut_size):
        r = random.uniform(0, adds_skills)
        temp_soma = 0
        for fit in fitness_array:
            temp_soma = temp_soma + fit[1]
            if temp_soma >= r:
                new_population.append(fit[0])
                break
    return new_population, fitness(stock_value, new_population, budget)

In [11]:
def stop_criterion(old_population, new_population, limit_to_converge):
    """
        :param: old_population - populacao ao iniciar a iteracao
        :param: new_population - populacao ao fim da iteracao
        :param: limit_to_converge - limiar abaixo do qual iremos considerar que ambas as pop convergem

        :return: True se for para parar, False c.c.
    """
    soma_old = 0
    for x in old_population:
        soma_old = soma_old + x[1]

    soma_new = 0
    for x in new_population:
        soma_new = soma_new + x[1]
    
    media_old = soma_old / len(old_population)
    media_new = soma_new / len(new_population)
    
    if abs(media_new - media_old) < limit_to_converge:
        return True
    else:
        return False

In [12]:
def generate_children(old_generation, crossover_function, crossover_rate):
    """
        :param: old_generation - 
        :param: crossover_function - 
        :param: crossover_rate - 
        
        :return: children - 
    """
    size_generation = len(old_generation)
    number_to_crossover = int(size_generation * crossover_rate)
    random.shuffle(old_generation)
    children = []
    for i in range (number_to_crossover):
        for j in range (i+1, number_to_crossover):
            new_chromossome1, new_chromossome2 = crossover_function(old_generation[i], old_generation[j])
            children.append(new_chromossome1)
            children.append(new_chromossome2)
    return children

In [13]:
def mutation_chromossome(chromossomes, mutation_function, mutation_rate):
    """
        :param: chromossomes - 
        :param: mutation_function - 
        :param: mutation_rate - 
        
        :return: chromossomes - 
    """
    number_chromossomes_to_mutate = int(len(chromossomes) * mutation_rate)
    random.shuffle(chromossomes)
    for i in range (0,number_chromossomes_to_mutate):
        chromossomes[i] = mutation_function(chromossomes[i])
    return chromossomes
    

In [14]:
def evolutionary_strategy1(stock_values, first_population, budget, crossover_function, delta, mutation_rate, crossover_rate, min_iteration_converge):
    """
        :param: stock_values - 
        :param: first_population - 
        :param: budget - 
        :param: crossover_function - 
        :param: delta -
        
        :return: old_population - 
    """
    flag = False
    iteration = 0
    old_population = first_population
    while (not flag):
        fitness_old_population = fitness(stock_values, old_population, budget)
        children = generate_children(first_population, crossover_function, crossover_rate)
        parents_and_children = old_population + children
        chromossomes_parents_children_mutated = mutation_chromossome(parents_and_children, mutation_v2, mutation_rate)
        new_population, fitness_new_population = selection(stock_values, chromossomes_parents_children_mutated, budget, len(old_population))
        flag = (stop_criterion(fitness_old_population, fitness_new_population, delta) and iteration > min_iteration_converge)
        iteration = iteration + 1
        old_population = new_population
    return old_population
        

In [15]:
def linear_normalization(fitness_population, increment=20):
    pop_size = len(fitness_population)
    fitness_population = sorted(fitness_population, key=lambda x: x[1], reverse = False)
    min_value = 1
    max_value = 201
    normalized_fitness = []
    for i in range(pop_size):
        temp_fit = min_value + ((max_value - min_value)/(pop_size - 1)) * (i)
        normalized_fitness.append([fitness_population[i][0],temp_fit])
    
    return normalized_fitness

In [16]:
def evolutionary_strategy2(stock_values, first_population, budget, crossover_function, mutation_func,delta, mutation_rate, crossover_rate, min_iteration_converge):
    """
        :param: stock_values - 
        :param: first_population - 
        :param: budget - 
        :param: crossover_function - 
        :param: mutation_func - 
        :param: delta -
        
        :return: old_population - 
    """
    flag = False
    iteration = 0
    old_population = first_population
    while (not flag):
        #         print("While: ", iteration)
        fitness_old_population = fitness(stock_values, old_population, budget)
        children = generate_children(first_population, crossover_function, crossover_rate)
        parents_and_children = old_population + children
        chromossomes_parents_children_mutated = mutation_chromossome(parents_and_children, mutation_func, mutation_rate)
        chromossomes_parents_children_mutated = random.sample(chromossomes_parents_children_mutated, len(chromossomes_parents_children_mutated))
        new_population, fitness_new_population = roulette_selection(stock_values, chromossomes_parents_children_mutated, budget, len(old_population))
        flag = (stop_criterion(fitness_old_population, fitness_new_population, delta) and iteration > min_iteration_converge) or iteration > 100
        iteration = iteration + 1
        old_population = new_population
    return old_population



# Let's play

## Primeiro conjunto de testes - Defuzzi: Risk - centroid Pct - LOM

### Variaveis que serão usadas para a primeira população
Para as ações da Alphabet serão testados:
1. Dados Fixos:
    1. Budget &rarr; 10.000,00
    2. Delta para considerar convergência &rarr; 0.01
2. Dados variáveis:
    1. População:
        1. Para uma população de 100 e outra de 1000, testamos:
            1. Duas funções de Crossover:
                1. Taxa de Crossover &rarr; 0.3
                2. Taxa de Crossover &rarr; 0.8
            2. Duas funções de Mutação:
                1. Taxa de Mutação &rarr; 0.3
                2. Taxa de Mutação &rarr; 0.8


#### Primeira iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.01
4. Crossover &rarr; crossover
    1. Taxa &rarr; 0.3
5. Mutação &rarr; mutation
    1. Taxa &rarr; 0.3
6. Melhor Resultado:
    [[0.029757806673384436, 16, 10, 339], 1.6809029999999998]
7. Pior Resultado:
    [[0.0903877154523101, 24, 3, 304], 0.0]
8. Média:
    0.3572531
9. Desvio:
    0.409838862518003

Dados fixos que serão usados para todas as iterações de 100 cromossomos

In [17]:
population_len = 10
budget = 10000
delta_to_converge = 0.01
min_iteration_converget = 3
np_array = excels[6].values # Irá utilizar a primeira lista de valores da bolsa
chromossomes = create_population(population_len)

Aplicação da estratégia evolucionária

In [18]:
# Dados variáveis
mutation_rate = 0.3
crossover_rate = 0.3

In [20]:
begin_time = time.time()
best_chromossomes_1 = evolutionary_strategy2(np_array, chromossomes, budget, crossover, mutation, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  103.55 segundos


#### Resultados da Primeira Iteração

In [21]:
tempo_1 = round(time.time() - begin_time, 2)
tempo_1

103.56

In [22]:
# best_chromossomes_1 = best_chromossomes_temp
best_chromossomes_1 = fitness(np_array, best_chromossomes_1, budget)
best_chromossomes_1

[[[0.024387026319011622, 22, 25, 28], 0.22503000000000012],
 [[0.04859978793630076, 22, 16, 160], 0.2071],
 [[0.04859978793630076, 22, 16, 160], 0.2071],
 [[0.046079438124175004, 1, 16, 383], 0.22172000000000008],
 [[0.051268761198829, 9, 17, 266], 0.2071],
 [[0.024387026319011622, 22, 25, 28], 0.22503000000000012],
 [[0.05748448119077433, 18, 5, 477], 0.026895999999999913],
 [[0.019407448994008005, 14, 24, 273], 0.21732600000000002],
 [[0.051268761198829, 9, 17, 266], 0.2071],
 [[0.019407448994008005, 14, 24, 273], 0.21732600000000002]]

In [23]:
best_chromossomes_1 = sorted(best_chromossomes_1, key=lambda x: x[1], reverse = False)
best_chromossomes_1

[[[0.05748448119077433, 18, 5, 477], 0.026895999999999913],
 [[0.04859978793630076, 22, 16, 160], 0.2071],
 [[0.04859978793630076, 22, 16, 160], 0.2071],
 [[0.051268761198829, 9, 17, 266], 0.2071],
 [[0.051268761198829, 9, 17, 266], 0.2071],
 [[0.019407448994008005, 14, 24, 273], 0.21732600000000002],
 [[0.019407448994008005, 14, 24, 273], 0.21732600000000002],
 [[0.046079438124175004, 1, 16, 383], 0.22172000000000008],
 [[0.024387026319011622, 22, 25, 28], 0.22503000000000012],
 [[0.024387026319011622, 22, 25, 28], 0.22503000000000012]]

Melhor cromossomo

In [24]:
best_1 = best_chromossomes_1[-1]
best_1

[[0.024387026319011622, 22, 25, 28], 0.22503000000000012]

Pior cromossomo

In [25]:
worst_1 = best_chromossomes_1[0]
worst_1

[[0.05748448119077433, 18, 5, 477], 0.026895999999999913]

Valor médio dos fitness

In [26]:
mean_1 = 0
for x in best_chromossomes_1: mean_1 = mean_1 + x[1]
mean_1 = mean_1/len(best_chromossomes_1)
mean_1

0.1961728

In [27]:
std_1 = 0
for x in best_chromossomes_1: std_1 = std_1 + (x[1] - mean_1) ** 2
std_1 = (std_1 / (len(best_chromossomes_1) - 1)) ** (1/2)
std_1

0.05994933135536676

#### Segunda iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.001
4. Crossover &rarr; crossover
    1. Taxa &rarr; 0.8
5. Mutação &rarr; mutation
    1. Taxa &rarr; 0.3

In [28]:
# Dados variáveis
mutation_rate = 0.3
crossover_rate = 0.8

In [29]:
begin_time = time.time()
best_chromossomes_2 = evolutionary_strategy2(np_array, chromossomes, budget, crossover, mutation, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  153.58 segundos


Resultado da segunda iteração

In [103]:
tempo_2 = 153.58
tempo_2

153.58

In [31]:
best_chromossomes_2 = fitness(np_array, best_chromossomes_2, budget)

In [32]:
best_chromossomes_2 = sorted(best_chromossomes_2, key=lambda x: x[1], reverse = False)

Melhor cromossomo

In [33]:
best_2 = best_chromossomes_2[-1]
best_2

[[0.04859978793630076, 22, 25, 354], 0.3264580000000002]

Pior cromossomo

In [34]:
worst_2 = best_chromossomes_2[0]
worst_2

[[0.04859978793630076, 22, 21, 28], 0.027184000000000014]

Valor médio dos fitness

In [35]:
mean_2 = 0
for x in best_chromossomes_2: mean_2 = mean_2 + x[1]
mean_2 = mean_2/len(best_chromossomes_2)
mean_2

0.17815560000000003

In [36]:
std_2 = 0
for x in best_chromossomes_2: std_2 = std_2 + (x[1] - mean_2) ** 2
std_2 = (std_2 / (len(best_chromossomes_2) - 1)) ** (1/2)
std_2

0.10646482022109988

#### Terceira iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.001
4. Crossover &rarr; crossover
    1. Taxa &rarr; 0.3
5. Mutação &rarr; mutation
    1. Taxa &rarr; 0.8

In [37]:
# Dados variáveis
mutation_rate = 0.8
crossover_rate = 0.3

In [39]:
begin_time = time.time()
best_chromossomes_3 = evolutionary_strategy2(np_array, chromossomes, budget, crossover, mutation, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  86.96 segundos


Resultado da terceira iteração

In [40]:
tempo_3 = round(time.time() - begin_time, 2)

In [110]:
tempo_9

299.69

In [41]:
best_chromossomes_3 = fitness(np_array, best_chromossomes_3, budget)

In [42]:
best_chromossomes_3 = sorted(best_chromossomes_3, key=lambda x: x[1], reverse = False)

Melhor cromossomo

In [43]:
best_3 = best_chromossomes_3[-1]
best_3

[[0.05192811268481813, 1, 24, 306], 0.2071]

Pior cromossomo

In [44]:
worst_3 = best_chromossomes_3[0]
worst_3

[[0.013584148004297227, 16, 11, 134], 0.09415900000000002]

Valor médio dos fitness

In [45]:
mean_3 = 0
for x in best_chromossomes_3: mean_3 = mean_3 + x[1]
mean_3 = mean_3/len(best_chromossomes_3)
mean_3

0.14719229999999997

In [46]:
std_3 = 0
for x in best_chromossomes_3: std_3 = std_3 + (x[1] - mean_3) ** 2
std_3 = (std_3 / (len(best_chromossomes_3) - 1)) ** (1/2)
std_3

0.03462094230712585

#### Quarta iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.001
4. Crossover &rarr; crossover2
    1. Taxa &rarr; 0.3
5. Mutação &rarr; mutation
    1. Taxa &rarr; 0.3

In [47]:
# Dados variáveis
mutation_rate = 0.3
crossover_rate = 0.3

In [48]:
begin_time = time.time()
best_chromossomes_4 = evolutionary_strategy2(np_array, chromossomes, budget, crossover2, mutation, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  166.02 segundos


In [49]:
tempo_4 = round(time.time() - begin_time, 2)

In [50]:
best_chromossomes_4 = fitness(np_array, best_chromossomes_4, budget)

In [51]:
best_chromossomes_4 = sorted(best_chromossomes_4, key=lambda x: x[1], reverse = False)

Melhor cromossomo

In [52]:
best_4 = best_chromossomes_4[-1]
best_4

[[0.049908325920641226, 11, 5, 450], 0.18339899999999998]

Pior cromossomo

In [53]:
worst_4 = best_chromossomes_4[0]
worst_4

[[0.019496509502395395, 29, 4, 231], 0.03639700000000012]

Valor médio dos fitness

In [54]:
mean_4 = 0
for x in best_chromossomes_4: mean_4 = mean_4 + x[1]
mean_4 = mean_4/len(best_chromossomes_4)
mean_4

0.13754910000000004

In [55]:
std_4 = 0
for x in best_chromossomes_4: std_4 = std_4 + (x[1] - mean_4) ** 2
std_4 = (std_4 / (len(best_chromossomes_4) - 1)) ** (1/2)
std_4

0.06566504036302552

#### Quinta iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.001
4. Crossover &rarr; crossover2
    1. Taxa &rarr; 0.8
5. Mutação &rarr; mutation
    1. Taxa &rarr; 0.3

In [56]:
# Dados variáveis
mutation_rate = 0.3
crossover_rate = 0.8

In [58]:
begin_time = time.time()
best_chromossomes_5 = evolutionary_strategy2(np_array, chromossomes, budget, crossover2, mutation, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  442.55 segundos


Resultado da quinta iteração

In [59]:
tempo_5 = round(time.time() - begin_time, 2)

In [60]:
best_chromossomes_5 = fitness(np_array, best_chromossomes_5, budget)

In [61]:
best_chromossomes_5 = sorted(best_chromossomes_5, key=lambda x: x[1], reverse = False)

Melhor cromossomo

In [62]:
best_5 = best_chromossomes_5[-1]
best_5

[[0.0477042104268528, 2, 30, 37], 0.2071]

Pior cromossomo

In [63]:
worst_5 = best_chromossomes_5[0]
worst_5

[[0.07392231270778872, 14, 25, 242], 0.030496000000000096]

Valor médio dos fitness

In [64]:
mean_5 = 0
for x in best_chromossomes_5: mean_5 = mean_5 + x[1]
mean_5 = mean_5/len(best_chromossomes_5)
mean_5

0.15108589999999997

In [65]:
std_5 = 0
for x in best_chromossomes_5: std_5 = std_5 + (x[1] - mean_5) ** 2
std_5 = (std_5 / (len(best_chromossomes_5) - 1)) ** (1/2)
std_5

0.0684914557111566

#### Sexta iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.001
4. Crossover &rarr; crossover2
    1. Taxa &rarr; 0.3
5. Mutação &rarr; mutation
    1. Taxa &rarr; 0.8

In [66]:
# Dados variáveis
mutation_rate = 0.8
crossover_rate = 0.3

In [67]:
begin_time = time.time()
best_chromossomes_6 = evolutionary_strategy2(np_array, chromossomes, budget, crossover2, mutation, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  288.24 segundos


Resultado da sexta iteração

In [68]:
tempo_6 = round(time.time() - begin_time, 2)

In [69]:
best_chromossomes_6 = fitness(np_array, best_chromossomes_6, budget)

In [70]:
best_chromossomes_6 = sorted(best_chromossomes_6, key=lambda x: x[1], reverse = False)

Melhor cromossomo

In [71]:
best_6 = best_chromossomes_6[-1]
best_6

[[0.06844099679841738, 23, 8, 175], 0.14282999999999993]

Pior cromossomo

In [72]:
worst_6 = best_chromossomes_6[0]
worst_6

[[0.06844099679841738, 22, 19, 229], 0.030496000000000096]

Valor médio dos fitness

In [73]:
mean_6 = 0
for x in best_chromossomes_6: mean_6 = mean_6 + x[1]
mean_6 = mean_6/len(best_chromossomes_6)
mean_6

0.1269251

In [74]:
std_6 = 0
for x in best_chromossomes_6: std_6 = std_6 + (x[1] - mean_6) ** 2
std_6 = (std_6 / (len(best_chromossomes_6) - 1)) ** (1/2)
std_6

0.03565939258674426

#### Sétima iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.001
4. Crossover &rarr; crossover
    1. Taxa &rarr; 0.3
5. Mutação &rarr; mutation_v2
    1. Taxa &rarr; 0.3

In [75]:
# Dados variáveis
mutation_rate = 0.3
crossover_rate = 0.3

In [76]:
begin_time = time.time()
best_chromossomes_7 = evolutionary_strategy2(np_array, chromossomes, budget, crossover, mutation_v2, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  102.47 segundos


Resultado da sétima iteração

In [77]:
tempo_7 = round(time.time() - begin_time, 2)

In [78]:
best_chromossomes_7 = fitness(np_array, best_chromossomes_7, budget)

In [79]:
best_chromossomes_7 = sorted(best_chromossomes_7, key=lambda x: x[1], reverse = False)

Melhor cromossomo

In [80]:
best_7 = best_chromossomes_7[-1]
best_7

[[0.039024865585914066, 11, 24, 411], 0.07430499999999993]

Pior cromossomo

In [81]:
worst_7 = best_chromossomes_7[0]
worst_7

[[0.06841443732086265, 1, 29, 163], 0.030496000000000096]

Valor médio dos fitness

In [82]:
mean_7 = 0
for x in best_chromossomes_7: mean_7 = mean_7 + x[1]
mean_7 = mean_7/len(best_chromossomes_7)
mean_7

0.05825580000000001

In [83]:
std_7 = 0
for x in best_chromossomes_7: std_7 = std_7 + (x[1] - mean_7) ** 2
std_7 = (std_7 / (len(best_chromossomes_7) - 1)) ** (1/2)
std_7

0.021152072295640353

#### Oitava iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.001
4. Crossover &rarr; crossover
    1. Taxa &rarr; 0.8
5. Mutação &rarr; mutation_v2
    1. Taxa &rarr; 0.3

In [84]:
# Dados variáveis
mutation_rate = 0.3
crossover_rate = 0.8

In [85]:
begin_time = time.time()
best_chromossomes_8 = evolutionary_strategy2(np_array, chromossomes, budget, crossover, mutation_v2, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  218.51 segundos


Resultado da oitava iteração

In [86]:
tempo_8 = round(time.time() - begin_time, 2)

In [87]:
best_chromossomes_8 = fitness(np_array, best_chromossomes_8, budget)

In [88]:
best_chromossomes_8 = sorted(best_chromossomes_8, key=lambda x: x[1], reverse = False)

Melhor cromossomo

In [89]:
best_8 = best_chromossomes_8[-1]
best_8

[[0.03381512981563863, 11, 5, 365], 0.17741499999999996]

Pior cromossomo

In [90]:
worst_8 = best_chromossomes_8[0]
worst_8

[[0.07236401539451241, 16, 22, 365], 0.030496000000000096]

Valor médio dos fitness

In [91]:
mean_8 = 0
for x in best_chromossomes_8: mean_8 = mean_8 + x[1]
mean_8 = mean_8/len(best_chromossomes_8)
mean_8

0.11503419999999995

In [92]:
std_8 = 0
for x in best_chromossomes_8: std_8 = std_8 + (x[1] - mean_8) ** 2
std_8 = (std_8 / (len(best_chromossomes_8) - 8)) ** (1/2)
std_8

0.1293870715519907

#### Nona iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.001
4. Crossover &rarr; crossover
    1. Taxa &rarr; 0.3
5. Mutação &rarr; mutation_v2
    1. Taxa &rarr; 0.8

In [93]:
# Dados variáveis
mutation_rate = 0.8
crossover_rate = 0.3

In [94]:
begin_time = time.time()
best_chromossomes_9 = evolutionary_strategy2(np_array, chromossomes, budget, crossover, mutation_v2, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  96.69 segundos


Resultado da nona iteração

In [95]:
tempo_9 = round(time.time() - begin_time, 2)

In [96]:
best_chromossomes_9 = fitness(np_array, best_chromossomes_9, budget)

In [97]:
best_chromossomes_9 = sorted(best_chromossomes_9, key=lambda x: x[1], reverse = False)

Melhor cromossomo

In [98]:
best_9 = best_chromossomes_9[-1]
best_9

[[0.02021508999242241, 17, 29, 45], 0.22274799999999995]

Pior cromossomo

In [99]:
worst_9 = best_chromossomes_9[0]
worst_9

[[0.023591393425030492, 18, 29, 294], 0.11475499999999993]

Valor médio dos fitness

In [100]:
mean_9 = 0
for x in best_chromossomes_9: mean_9 = mean_9 + x[1]
mean_9 = mean_9/len(best_chromossomes_9)
mean_9

0.16172760000000003

In [101]:
std_9 = 0
for x in best_chromossomes_9: std_9 = std_9 + (x[1] - mean_9) ** 2
std_9 = (std_9 / (len(best_chromossomes_9) - 1)) ** (1/2)
std_9

0.04391272164241749

## Segundo conjunto de testes - Defuzzi: Risk/Pct - Centroid

### Variaveis que serão usadas para a primeira população
Para as ações da Alphabet serão testados:
1. Dados Fixos:
    1. Budget &rarr; 10.000,00
    2. Delta para considerar convergência &rarr; 0.01
2. Dados variáveis:
    1. População:
        1. Para uma população de 100 e outra de 1000, testamos:
            1. Duas funções de Crossover:
                1. Taxa de Crossover &rarr; 0.3
                2. Taxa de Crossover &rarr; 0.8
            2. Duas funções de Mutação:
                1. Taxa de Mutação &rarr; 0.3
                2. Taxa de Mutação &rarr; 0.8


#### Primeira iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.01
4. Crossover &rarr; crossover
    1. Taxa &rarr; 0.3
5. Mutação &rarr; mutation
    1. Taxa &rarr; 0.3
6. Melhor Resultado:
    [[0.029757806673384436, 16, 10, 339], 1.6809029999999998]
7. Pior Resultado:
    [[0.0903877154523101, 24, 3, 304], 0.0]
8. Média:
    0.3572531
9. Desvio:
    0.409838862518003

Dados fixos que serão usados para todas as iterações de 100 cromossomos

In [18]:
population_len = 10
budget = 10000
delta_to_converge = 0.01
min_iteration_converget = 3
np_array = excels[6].values # Irá utilizar a primeira lista de valores da bolsa
chromossomes = create_population(population_len)

Aplicação da estratégia evolucionária

In [19]:
# Dados variáveis
mutation_rate = 0.3
crossover_rate = 0.3

In [20]:
begin_time = time.time()
best_chromossomes_1 = evolutionary_strategy2(np_array, chromossomes, budget, crossover, mutation, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  68.92 segundos


#### Resultados da Primeira Iteração

In [21]:
tempo_1 = 68.92
tempo_1

68.92

In [22]:
# best_chromossomes_1 = best_chromossomes_temp
best_chromossomes_1 = fitness(np_array, best_chromossomes_1, budget)
best_chromossomes_1

[[[0.05497170765277004, 2, 16, 431], 0.19897000000000006],
 [[0.03333103317732289, 27, 14, 313], 0.2208960000000003],
 [[0.05497170765277004, 2, 16, 431], 0.19897000000000006],
 [[0.03333103317732289, 27, 14, 313], 0.2208960000000003],
 [[0.05497170765277004, 2, 16, 431], 0.19897000000000006],
 [[0.05497170765277004, 2, 16, 431], 0.19897000000000006],
 [[0.050173952817343076, 22, 27, 321], 0.1885620000000001],
 [[0.05497170765277004, 2, 16, 431], 0.19897000000000006],
 [[0.05497170765277004, 2, 16, 431], 0.19897000000000006],
 [[0.0559867315560339, 9, 8, 263], 0.2028100000000002]]

In [23]:
best_chromossomes_1 = sorted(best_chromossomes_1, key=lambda x: x[1], reverse = False)
best_chromossomes_1

[[[0.050173952817343076, 22, 27, 321], 0.1885620000000001],
 [[0.05497170765277004, 2, 16, 431], 0.19897000000000006],
 [[0.05497170765277004, 2, 16, 431], 0.19897000000000006],
 [[0.05497170765277004, 2, 16, 431], 0.19897000000000006],
 [[0.05497170765277004, 2, 16, 431], 0.19897000000000006],
 [[0.05497170765277004, 2, 16, 431], 0.19897000000000006],
 [[0.05497170765277004, 2, 16, 431], 0.19897000000000006],
 [[0.0559867315560339, 9, 8, 263], 0.2028100000000002],
 [[0.03333103317732289, 27, 14, 313], 0.2208960000000003],
 [[0.03333103317732289, 27, 14, 313], 0.2208960000000003]]

Melhor cromossomo

In [24]:
best_1 = best_chromossomes_1[-1]
best_1

[[0.03333103317732289, 27, 14, 313], 0.2208960000000003]

Pior cromossomo

In [25]:
worst_1 = best_chromossomes_1[0]
worst_1

[[0.050173952817343076, 22, 27, 321], 0.1885620000000001]

Valor médio dos fitness

In [26]:
mean_1 = 0
for x in best_chromossomes_1: mean_1 = mean_1 + x[1]
mean_1 = mean_1/len(best_chromossomes_1)
mean_1

0.2026984000000001

In [27]:
std_1 = 0
for x in best_chromossomes_1: std_1 = std_1 + (x[1] - mean_1) ** 2
std_1 = (std_1 / (len(best_chromossomes_1) - 1)) ** (1/2)
std_1

0.010249993769103882

#### Segunda iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.001
4. Crossover &rarr; crossover
    1. Taxa &rarr; 0.8
5. Mutação &rarr; mutation
    1. Taxa &rarr; 0.3

In [28]:
# Dados variáveis
mutation_rate = 0.3
crossover_rate = 0.8

In [29]:
begin_time = time.time()
best_chromossomes_2 = evolutionary_strategy2(np_array, chromossomes, budget, crossover, mutation, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  228.94 segundos


Resultado da segunda iteração

In [30]:
tempo_2 = 228.94
tempo_2

228.94

In [31]:
best_chromossomes_2 = fitness(np_array, best_chromossomes_2, budget)

In [33]:
best_chromossomes_2 = sorted(best_chromossomes_2, key=lambda x: x[1], reverse = False)

Melhor cromossomo

In [34]:
best_2 = best_chromossomes_2[-1]
best_2

[[0.013732684419469822, 2, 20, 313], 0.21384400000000006]

Pior cromossomo

In [35]:
worst_2 = best_chromossomes_2[0]
worst_2

[[0.06390727324323515, 1, 12, 450], 0.022872000000000118]

Valor médio dos fitness

In [36]:
mean_2 = 0
for x in best_chromossomes_2: mean_2 = mean_2 + x[1]
mean_2 = mean_2/len(best_chromossomes_2)
mean_2

0.14594820000000006

In [37]:
std_2 = 0
for x in best_chromossomes_2: std_2 = std_2 + (x[1] - mean_2) ** 2
std_2 = (std_2 / (len(best_chromossomes_2) - 1)) ** (1/2)
std_2

0.0556858831877523

#### Terceira iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.001
4. Crossover &rarr; crossover
    1. Taxa &rarr; 0.3
5. Mutação &rarr; mutation
    1. Taxa &rarr; 0.8

In [38]:
# Dados variáveis
mutation_rate = 0.8
crossover_rate = 0.3

In [39]:
begin_time = time.time()
best_chromossomes_3 = evolutionary_strategy2(np_array, chromossomes, budget, crossover, mutation, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  67.53 segundos


Resultado da terceira iteração

In [40]:
tempo_3 = 67.53

In [42]:
best_chromossomes_3 = fitness(np_array, best_chromossomes_3, budget)

In [43]:
best_chromossomes_3 = sorted(best_chromossomes_3, key=lambda x: x[1], reverse = False)

Melhor cromossomo

In [44]:
best_3 = best_chromossomes_3[-1]
best_3

[[0.025258102732316795, 2, 28, 254], 0.18272800000000006]

Pior cromossomo

In [45]:
worst_3 = best_chromossomes_3[0]
worst_3

[[0.0652253745315831, 8, 22, 439], 0.022872000000000118]

Valor médio dos fitness

In [46]:
mean_3 = 0
for x in best_chromossomes_3: mean_3 = mean_3 + x[1]
mean_3 = mean_3/len(best_chromossomes_3)
mean_3

0.13147330000000007

In [47]:
std_3 = 0
for x in best_chromossomes_3: std_3 = std_3 + (x[1] - mean_3) ** 2
std_3 = (std_3 / (len(best_chromossomes_3) - 1)) ** (1/2)
std_3

0.05224320113139652

#### Quarta iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.001
4. Crossover &rarr; crossover2
    1. Taxa &rarr; 0.3
5. Mutação &rarr; mutation
    1. Taxa &rarr; 0.3

In [48]:
# Dados variáveis
mutation_rate = 0.3
crossover_rate = 0.3

In [49]:
begin_time = time.time()
best_chromossomes_4 = evolutionary_strategy2(np_array, chromossomes, budget, crossover2, mutation, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  62.34 segundos


In [50]:
tempo_4 = 62.34

In [51]:
best_chromossomes_4 = fitness(np_array, best_chromossomes_4, budget)

In [52]:
best_chromossomes_4 = sorted(best_chromossomes_4, key=lambda x: x[1], reverse = False)

Melhor cromossomo

In [53]:
best_4 = best_chromossomes_4[-1]
best_4

[[0.05497170765277004, 26, 16, 199], 0.1468600000000002]

Pior cromossomo

In [54]:
worst_4 = best_chromossomes_4[0]
worst_4

[[0.024345923863482458, 22, 3, 254], 0.08910600000000013]

Valor médio dos fitness

In [55]:
mean_4 = 0
for x in best_chromossomes_4: mean_4 = mean_4 + x[1]
mean_4 = mean_4/len(best_chromossomes_4)
mean_4

0.12751050000000028

In [56]:
std_4 = 0
for x in best_chromossomes_4: std_4 = std_4 + (x[1] - mean_4) ** 2
std_4 = (std_4 / (len(best_chromossomes_4) - 1)) ** (1/2)
std_4

0.02239980548154626

#### Quinta iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.001
4. Crossover &rarr; crossover2
    1. Taxa &rarr; 0.8
5. Mutação &rarr; mutation
    1. Taxa &rarr; 0.3

In [57]:
# Dados variáveis
mutation_rate = 0.3
crossover_rate = 0.8

In [58]:
begin_time = time.time()
best_chromossomes_5 = evolutionary_strategy2(np_array, chromossomes, budget, crossover2, mutation, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  444.98 segundos


Resultado da quinta iteração

In [59]:
tempo_5 = 444.98

In [60]:
best_chromossomes_5 = fitness(np_array, best_chromossomes_5, budget)

In [61]:
best_chromossomes_5 = sorted(best_chromossomes_5, key=lambda x: x[1], reverse = False)

Melhor cromossomo

In [63]:
best_5 = best_chromossomes_5[-1]
best_5

[[0.014527123198729876, 24, 27, 282], 0.19364300000000004]

Pior cromossomo

In [64]:
worst_5 = best_chromossomes_5[0]
worst_5

[[0.08499958111170343, 14, 24, 87], 0.01981800000000003]

Valor médio dos fitness

In [65]:
mean_5 = 0
for x in best_chromossomes_5: mean_5 = mean_5 + x[1]
mean_5 = mean_5/len(best_chromossomes_5)
mean_5

0.12444380000000006

In [66]:
std_5 = 0
for x in best_chromossomes_5: std_5 = std_5 + (x[1] - mean_5) ** 2
std_5 = (std_5 / (len(best_chromossomes_5) - 1)) ** (1/2)
std_5

0.054120492951273885

#### Sexta iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.001
4. Crossover &rarr; crossover2
    1. Taxa &rarr; 0.3
5. Mutação &rarr; mutation
    1. Taxa &rarr; 0.8

In [67]:
# Dados variáveis
mutation_rate = 0.8
crossover_rate = 0.3

In [68]:
begin_time = time.time()
best_chromossomes_6 = evolutionary_strategy2(np_array, chromossomes, budget, crossover2, mutation, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  136.27 segundos


Resultado da sexta iteração

In [69]:
tempo_6 = 136.27

In [70]:
best_chromossomes_6 = fitness(np_array, best_chromossomes_6, budget)

In [71]:
best_chromossomes_6 = sorted(best_chromossomes_6, key=lambda x: x[1], reverse = False)

Melhor cromossomo

In [72]:
best_6 = best_chromossomes_6[-1]
best_6

[[0.04156247430979256, 21, 20, 143], 0.18805800000000017]

Pior cromossomo

In [73]:
worst_6 = best_chromossomes_6[0]
worst_6

[[0.08191657100538824, 20, 14, 144], 0.022872000000000118]

Valor médio dos fitness

In [74]:
mean_6 = 0
for x in best_chromossomes_6: mean_6 = mean_6 + x[1]
mean_6 = mean_6/len(best_chromossomes_6)
mean_6

0.09641830000000012

In [75]:
std_6 = 0
for x in best_chromossomes_6: std_6 = std_6 + (x[1] - mean_6) ** 2
std_6 = (std_6 / (len(best_chromossomes_6) - 1)) ** (1/2)
std_6

0.0683887459813382

#### Sétima iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.001
4. Crossover &rarr; crossover
    1. Taxa &rarr; 0.3
5. Mutação &rarr; mutation_v2
    1. Taxa &rarr; 0.3

In [76]:
# Dados variáveis
mutation_rate = 0.3
crossover_rate = 0.3

In [77]:
begin_time = time.time()
best_chromossomes_7 = evolutionary_strategy2(np_array, chromossomes, budget, crossover, mutation_v2, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  147.09 segundos


Resultado da sétima iteração

In [79]:
tempo_7 = 147.09

In [80]:
best_chromossomes_7 = fitness(np_array, best_chromossomes_7, budget)

In [81]:
best_chromossomes_7 = sorted(best_chromossomes_7, key=lambda x: x[1], reverse = False)

Melhor cromossomo

In [82]:
best_7 = best_chromossomes_7[-1]
best_7

[[0.013732684419469822, 7, 11, 374], 0.14247700000000021]

Pior cromossomo

In [83]:
worst_7 = best_chromossomes_7[0]
worst_7

[[0.050173952817343076, 24, 15, 298], 0.11473000000000029]

Valor médio dos fitness

In [84]:
mean_7 = 0
for x in best_chromossomes_7: mean_7 = mean_7 + x[1]
mean_7 = mean_7/len(best_chromossomes_7)
mean_7

0.12599900000000017

In [85]:
std_7 = 0
for x in best_chromossomes_7: std_7 = std_7 + (x[1] - mean_7) ** 2
std_7 = (std_7 / (len(best_chromossomes_7) - 1)) ** (1/2)
std_7

0.010108929364125984

#### Oitava iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.001
4. Crossover &rarr; crossover
    1. Taxa &rarr; 0.8
5. Mutação &rarr; mutation_v2
    1. Taxa &rarr; 0.3

In [86]:
# Dados variáveis
mutation_rate = 0.3
crossover_rate = 0.8

In [87]:
begin_time = time.time()
best_chromossomes_8 = evolutionary_strategy2(np_array, chromossomes, budget, crossover, mutation_v2, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  330.69 segundos


Resultado da oitava iteração

In [89]:
tempo_8 = 330.69

In [90]:
best_chromossomes_8 = fitness(np_array, best_chromossomes_8, budget)

In [91]:
best_chromossomes_8 = sorted(best_chromossomes_8, key=lambda x: x[1], reverse = False)

Melhor cromossomo

In [92]:
best_8 = best_chromossomes_8[-1]
best_8

[[0.04276160301445853, 4, 22, 145], 0.1590880000000001]

Pior cromossomo

In [93]:
worst_8 = best_chromossomes_8[0]
worst_8

[[0.0762399053752945, 5, 23, 374], 0.022872000000000118]

Valor médio dos fitness

In [94]:
mean_8 = 0
for x in best_chromossomes_8: mean_8 = mean_8 + x[1]
mean_8 = mean_8/len(best_chromossomes_8)
mean_8

0.11518200000000005

In [95]:
std_8 = 0
for x in best_chromossomes_8: std_8 = std_8 + (x[1] - mean_8) ** 2
std_8 = (std_8 / (len(best_chromossomes_8) - 8)) ** (1/2)
std_8

0.09385162455706357

#### Nona iteração:
1. População &rarr; 100
2. Budget &rarr; 10.000
3. Delta &rarr; 0.001
4. Crossover &rarr; crossover
    1. Taxa &rarr; 0.3
5. Mutação &rarr; mutation_v2
    1. Taxa &rarr; 0.8

In [96]:
# Dados variáveis
mutation_rate = 0.8
crossover_rate = 0.3

In [97]:
begin_time = time.time()
best_chromossomes_9 = evolutionary_strategy2(np_array, chromossomes, budget, crossover, mutation_v2, delta_to_converge, mutation_rate, crossover_rate, min_iteration_converget)
print ("Tempo para convergir: ", round(time.time() - begin_time, 2), "segundos")

Tempo para convergir:  109.46 segundos


Resultado da nona iteração

In [98]:
tempo_9 = 109.46

In [99]:
best_chromossomes_9 = fitness(np_array, best_chromossomes_9, budget)

In [100]:
best_chromossomes_9 = sorted(best_chromossomes_9, key=lambda x: x[1], reverse = False)

Melhor cromossomo

In [101]:
best_9 = best_chromossomes_9[-1]
best_9

[[0.05123513294149655, 4, 19, 98], 0.18103800000000012]

Pior cromossomo

In [102]:
worst_9 = best_chromossomes_9[0]
worst_9

[[0.012466142985667016, 2, 2, 384], 0.07552700000000132]

Valor médio dos fitness

In [103]:
mean_9 = 0
for x in best_chromossomes_9: mean_9 = mean_9 + x[1]
mean_9 = mean_9/len(best_chromossomes_9)
mean_9

0.14600170000000018

In [104]:
std_9 = 0
for x in best_chromossomes_9: std_9 = std_9 + (x[1] - mean_9) ** 2
std_9 = (std_9 / (len(best_chromossomes_9) - 1)) ** (1/2)
std_9

0.030811068386575005